In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from blueqat import opt
%matplotlib inline

ImportError: cannot import name 'opt'

In [15]:
import blueqat as bq

In [ ]:
a = bq.

最小化対象:  
$ \displaystyle　\sum_{ i = 1 }^{ N } w_i x_i,　x_i \in \{0, 1\} $
<br>
<br>
subject to:  
$ \displaystyle　\sum_{ i = 1 }^{ N } u_i x_i \geq U $
<br>
<br>
$\implies$最小化対象:  
$ \displaystyle　\sum_{ i = 1 }^{ N } w_i x_i + \alpha (\sum_{ i = 1 }^{ N } u_i x_i - U)^2 $
<br>
<br>
$\implies$最小化対象（変形後）:  
$ \displaystyle　\mathbf{x}^T \mathbf{w} + \alpha\{\mathbf{x}^T \mathbf{u} - (5 + \lambda_{1} + \lambda_{2} + \lambda_{3})\}^2,　\lambda_j \in \{0, 1\}  $  
<br>
※「高々最低取得単位数よりも3多い程度に収まるだろう」という想定のもと、λを3個に制限

最適化対象の式に対して、  
$ \mathbf{P} = \mathbf{x}^T C \mathbf{x} $  
の形になるよう式変形を行う
<br>
<br>
$$
    \mathbf{C} =
        \left(\begin{array}{ccc}
            w_{1} + \alpha u_{1}^2 - 10u_{1}& 2\alpha u_{1}u_{2} & 0 & \cdots & 0 & -2\alpha u_{1} & \cdots　& & -2\alpha u_{1} \\
            & \ddots & 2\alpha u_{2}u_{3} & \ddots & \vdots & \vdots & & & \vdots \\
            & & \ddots & & 0 \\
            & & & \ddots & 2\alpha u_{N-1}u_{N}\\
            & & & &  w_{N}+\alpha u_{N}^2 - 10u_{N} & -2\alpha u_{N} & \cdots　& & -2\alpha u_{N}  \\
            & & & & & 11\alpha & 2\alpha & \cdots & 2\alpha \\
            & & & & & & 11\alpha & & \vdots \\
            & & & & & & & \ddots & 2\alpha\\
            & & & & & & & & 11\alpha   \\
        \end{array}\right)
$$

In [21]:
# u_i : 科目iの履修による取得単位数
u = [1, 3, 2, 2, 1]

# w_i : 科目iの単位取得難易度＝単位取得所要時間（工数）
w = [1, 5, 3, 1, 2]

# 行列サイズ
N = len(u)
# 必要取得単位数 : 5で固定
# 最低取得単位数+3まで許容する
U = 3
# 最適化パラメータα
alpha = 5

In [22]:
def get_qubo(u: list,w: list, N: int, U: int):
    
    # 行列の合計サイズ
    size = N + U
    qubo = np.zeros((size, size))
    
    # 1. x vector part
    # 対角成分
    for i in range(N):
        for j in range(N):
            if j >= i + 1:
                qubo[i][j] = w[i] + alpha * (u[i] ** 2) - 10 * alpha * u[i]
   # 上三角行列部分
    for i in range(N):
        for j in range(N):
            if j == i + 1:
                qubo[i][j] = 2 * alpha * u[i] * u[i + 1]
    for i in range(N):
        for j in range(N, N + U):
            qubo[i][j] = -2 * alpha * u[i]
                
    # 2. lambda vector part
    # 対角成分
    for i in range(N, N + U):
        for j in range(N, N + U):
            if j == i:
                qubo[i][j] = 11 * alpha
    # 上三角行列部分
    for i in range(N, N + U):
        for j in range(N, N + U):
            if j >= i + 1:
                qubo[i][j] = 2 * alpha
                
    return qubo

In [23]:
qubo = get_qubo(u, w, N, U)
qubo

array([[ -89.,   60.,    0.,    0.,    0.,  -20.,  -20.,  -20.],
       [   0., -205.,  120.,    0.,    0.,  -60.,  -60.,  -60.],
       [   0.,    0., -157.,   80.,    0.,  -40.,  -40.,  -40.],
       [   0.,    0.,    0., -159.,   40.,  -40.,  -40.,  -40.],
       [   0.,    0.,    0.,    0.,  -88.,  -20.,  -20.,  -20.],
       [   0.,    0.,    0.,    0.,    0.,  110.,   20.,   20.],
       [   0.,    0.,    0.,    0.,    0.,    0.,  110.,   20.],
       [   0.,    0.,    0.,    0.,    0.,    0.,    0.,  110.]])

In [24]:
a = opt.opt() 
a.qubo = qubo
result = a.sa(shots = 100, sampler = "fast")
opt.counter(result)

Counter({'11111111': 100})

ってこれ、、、最大化じゃなくて最小化問題だからナップサック問題向けのQUBO解法がそのまま使えるわけじゃないじゃん、、、